In [1]:
from selenium import webdriver
import pandas as pd
from scrap_article_new_tab import scrap_article
# from threading import Thread

from datetime import datetime as dt
from datetime import timedelta as td
from os import makedirs
from os.path import exists
from time import sleep
from json import loads

In [2]:
now = dt.now()
year = now.year
month = now.month
day = now.day
ts = str(int(round(now.timestamp())))

In [3]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
chrome = webdriver.Chrome(options=options)

In [4]:
# chrome = webdriver.Chrome()#chrome = webdriver.Chrome()

In [5]:
chrome.get(url = 'http://wyborcza.pl/0,75248.html')

In [6]:
entries = []
for entry in chrome\
    .find_element_by_xpath('//*[@id="pagetype_index"]/section[5]/div/main/div/div[2]/div/div[2]/ul')\
    .find_elements_by_class_name('entry'):
    entryContainer = {}
    
    entryContainer['author'] = entry.find_element_by_class_name('author').text.replace('|', '').strip()
    header = entry.find_element_by_tag_name('h3')\
                .find_element_by_tag_name('a')
    entryContainer['title'] = header.get_attribute('title')
    entryContainer['url'] = header.get_attribute('href')
    entryContainer['article_id'] = entryContainer['url'].split('/')[-1].split(',')[2]
    entryContainer['lead'] = entry.find_element_by_tag_name('p').text
    entryContainer['date'] = entry.find_element_by_class_name('when').text
    
    entries.append(entryContainer)

In [7]:
# try to click rodo (first click opens sometimes an ad)
for i in range(3):
    try:
        sleep(3)
        chrome.find_element_by_xpath('//*[@id="rodoNotificationWrapper"]/div[1]/div/p[2]')\
            .click()
    except:
        pass

In [8]:
# close new window
try:
    chrome.switch_to.window(chrome.window_handles[1])
    chrome.close()
except:
    pass

In [9]:
scrapped_articles = pd.DataFrame(entries)
scrapped_articles['is_new'] = True
scrapped_articles['content'] = None

In [10]:
#scrapped_articles.head(5)

In [11]:
existing_articles = pd.read_csv('articles.csv')

In [12]:
#existing_articles.head(5)

In [13]:
len(scrapped_articles[~scrapped_articles['article_id'].isin(existing_articles['article_id'].unique())])

25

In [14]:
len(existing_articles)

75

In [15]:
combined_articles = existing_articles.append(scrapped_articles, sort=False)

In [16]:
len(combined_articles)

100

In [17]:
all_articles = combined_articles.drop_duplicates(keep='first')

In [18]:
all_articles['date'] = pd.to_datetime(all_articles['date'])

In [19]:
articles_to_drop = all_articles[
    all_articles['date'] <= dt.now() - td(days=7)
]

In [20]:
if articles_to_drop.empty == True:
    pass
else:
    existing_articles_to_drop = pd.read_csv('old_articles.csv')
    existing_articles_to_drop = existing_articles_to_drop.append(articles_to_drop)
    # backup file
    filename_to_save = 'data/backup/old/{}/{}/{}/'.format(
        year, month, day
    )
    if not exists(filename_to_save):
        makedirs(filename_to_save)
    existing_articles_to_drop.to_csv(filename_to_save+str(ts)+'.csv', index=False)
    # save used in script file
    existing_articles_to_drop.to_csv('data/old_articles.csv', index=False)

In [21]:
articles_to_continue = all_articles[
    all_articles['date'] > dt.now() - td(days=7)
]

In [22]:
#  --------------------- TMP ---------------------
articles_to_continue['is_new'] = True
articles_to_continue['last_date'] = dt.now() - td(minutes=60)

In [23]:
#articles_to_continue.head(1)

In [24]:
def check_date(last_date, pub_date):
    hours_since_pub = (dt.now() - pub_date).total_seconds() / 60 / 60
    if hours_since_pub < 2:
        interval = 5 # in minutes
    elif hours_since_pub < 5:
        interval = 10
    elif hours_since_pub < 10:
        interval = 20
    else:
        interval = 30
    
    if dt.now() - td(minutes=interval) < last_date:
        return True
    else:
        return False

In [25]:
def check_date(pub_date):
    days_since_pub = (dt.now() - pub_date).total_seconds() / 60 / 60 / 24
    if days_since_pub > 5:
        return True
    else:
        return False

In [26]:
articles_to_scrap = articles_to_continue[articles_to_continue['date'] < dt.now() - td(days=5)]

In [27]:
articles_to_continue[articles_to_continue['date'] >= dt.now() - td(days=5)].to_csv('articles.csv', index = False)

In [28]:
articles_list = articles_to_scrap.to_dict('rows')

In [29]:
# TMP
#options = webdriver.ChromeOptions()
#options.add_argument("--disable-popup-blocking")
#chrome = webdriver.Chrome()



In [30]:
#chrome.execute_script("window.close();")

In [31]:
#chrome.close()

In [32]:
# chrome.execute_script("window.open('{}','_blank');".format(article['url']))
# for i in range(10):
#     if chrome.execute_script("return document.readyState") != 'complete':
#         print('not ready')

In [33]:
#chrome.switch_to.window(chrome.window_handles[0])

In [34]:
chrome.switch_to.window(chrome.window_handles[0])

In [35]:
container = []
first = True
if len(articles_list) != 0:
    for article in articles_list:
        article_id = article['url'].split('/')[-1].split(',')[2]
        chrome.execute_script("window.open('{}','_blank');".format(article['url']))
        chrome.close()
        chrome.switch_to.window(chrome.window_handles[0])
        sleep(7)



        if check_date(article['date']):
            if article['is_new']:
                a = loads(scrap_article(chrome=chrome, first_time=True))
                article['article_id'] = article_id
                article['content'] = a['content']
                article['division'] = a['division']
                article['highlight'] = a['highlight']
                article['media_type'] = a['media_type']
                article['media_src'] = a['media_src']
                article['media_desc'] = a['media_desc']
                article['is_new'] = False
                with open('data/comments/'+article_id, 'w') as f:
                    f.write(str({'row': {
                       'comments': a['comments'],
                       'timestamp': dt.timestamp(dt.now())
                       }
                    }))
        container.append(article)


In [36]:
if len(articles_list) != 0:
    articles_updated = pd.DataFrame(container)
    if 'Unnamed: 0' in articles_updated.columns:
        articles_updated = articles_updated.drop('Unnamed: 0', axis=1)

    filename_to_save = 'data/backup/scrapped/{}/{}/{}'.format(
            year, month, day, ts
        )
    if not exists(filename_to_save):
        makedirs(filename_to_save)

    articles_updated.to_csv('data/backup/scrapped/{}/{}/{}/{}.csv'.format(
            year, month, day, ts
        )
    )
    
    